In [9]:
# import chart_studio.plotly as py
import plotly.graph_objects as go
import json
import pandas as pd

from utils import calculate_nutripoints

In [10]:
# Geojson data source:
# https://github.com/ft-interactive/geo-data/blob/master/uk/london/london-wards-2014.geojson

GEO_DATA_PATH = 'data/london-wards-2014.geojson'

with open(GEO_DATA_PATH) as f:
    geo_london = json.load(f)

year_grocery = pd.read_csv("data/year_osward_grocery.csv")

print(year_grocery.columns.tolist())

min_grocery = year_grocery[['energy_tot', 'saturate', 'salt',
                            'sugar', 'f_fruit_veg', 'fibre', 'protein']].min(axis=0)

max_grocery = year_grocery[['energy_tot', 'saturate', 'salt',
                            'sugar', 'f_fruit_veg', 'fibre', 'protein']].max(axis=0)

year_grocery["nutripoints"] = year_grocery.apply(
    lambda row: calculate_nutripoints(row, min_grocery, max_grocery), axis=1)

wellbeing_scores = pd.read_excel(
    "data/london-ward-well-being-probability-scores.xls", sheet_name="Scores", header=[0, 1])
wellbeing_scores = wellbeing_scores.loc[:, (slice(
    None), [2013, "New ward code", "Ward name", "Borough"])].dropna(how="all")
wellbeing_scores = wellbeing_scores.droplevel(1, axis=1)

wellbeing_grocery = pd.merge(
    left=year_grocery, right=wellbeing_scores, left_on='area_id', right_on="New ward code")

LONDON_CENTER_COORD = {"lat": 51.5074, "lon": -0.118092}

['area_id', 'weight', 'weight_perc2.5', 'weight_perc25', 'weight_perc50', 'weight_perc75', 'weight_perc97.5', 'weight_std', 'weight_ci95', 'volume', 'volume_perc2.5', 'volume_perc25', 'volume_perc50', 'volume_perc75', 'volume_perc97.5', 'volume_std', 'volume_ci95', 'fat', 'fat_perc2.5', 'fat_perc25', 'fat_perc50', 'fat_perc75', 'fat_perc97.5', 'fat_std', 'fat_ci95', 'saturate', 'saturate_perc2.5', 'saturate_perc25', 'saturate_perc50', 'saturate_perc75', 'saturate_perc97.5', 'saturate_std', 'saturate_ci95', 'salt', 'salt_perc2.5', 'salt_perc25', 'salt_perc50', 'salt_perc75', 'salt_perc97.5', 'salt_std', 'salt_ci95', 'sugar', 'sugar_perc2.5', 'sugar_perc25', 'sugar_perc50', 'sugar_perc75', 'sugar_perc97.5', 'sugar_std', 'sugar_ci95', 'protein', 'protein_perc2.5', 'protein_perc25', 'protein_perc50', 'protein_perc75', 'protein_perc97.5', 'protein_std', 'protein_ci95', 'carb', 'carb_perc2.5', 'carb_perc25', 'carb_perc50', 'carb_perc75', 'carb_perc97.5', 'carb_std', 'carb_ci95', 'fibre', 'fi

In [11]:
FEATURES = ['representativeness_norm', 'fibre',
            'protein', 'fat', 'carb', 'salt', 'h_nutrients_calories', 'nutripoints']

map_plots = []

for f in FEATURES:
    map_plots.append(go.Choroplethmapbox(
        geojson=geo_london,
        locations=wellbeing_grocery['area_id'],
        featureidkey="properties.gss_code_ward",
        text=wellbeing_grocery['Ward name'],
        z=wellbeing_grocery[f],
        colorbar=dict(thickness=20, ticklen=3),
        marker_line_width=0, marker_opacity=0.7,
        visible=False,
        hovertemplate="<b>ward</b>: <b>%{text}</b><br><b>val</b>: %{z}<br><extra></extra>"
    ))

map_plots[0]['visible'] = True

In [12]:
layout = go.Layout(
    autosize=True,
    mapbox=dict(
        style="carto-positron",
        center=LONDON_CENTER_COORD,
        zoom=8.7),
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor='rgb(204, 204, 204)',
    plot_bgcolor='rgb(204, 204, 204)',
)

In [13]:
# add a dropdown menu in the layout
layout.update(
    updatemenus=[
        dict(x=0.005,
             y=0.95,
             xanchor='left',
             yanchor='middle',
             buttons=[
                 dict(
                     args=['visible', [True, False, False, False,
                                       False, False, False, False]],
                     label='Representativeness',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, True, False, False,
                                       False, False, False, False]],
                     label='Fibre',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, True, False,
                                       False, False, False, False]],
                     label='Protein',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False, False,
                                       True, False, False, False]],
                     label='Fat',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False, False,
                                       False, True, False, False]],
                     label='Carbs',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False, False,
                                       False, False, True, False]],
                     label='Salt',
                     method='restyle'
                 ),
                dict(
                     args=['visible', [False, False, False, False,
                                       False, False, True, False]],
                     label='Entropy',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False, False,
                                       False, False, False, True]],
                     label='Nutripoints',
                     method='restyle'
                 )
             ])
    ]
);

In [14]:
fig = go.Figure(data=map_plots, layout=layout)

# py.plot(figure_or_data=fig, filename='tesco_map', auto_open=True, sharing='secret')
fig.write_html("map.html")